```
네, 드디어 모든 환경 설정의 결실을 볼 시간입니다. KoGPT2 모델을 '노무현 전 대통령'의 연설문 데이터로 파인튜닝(Fine-tuning)하여, 해당 인물의 말투와 스타일을 가진 '나만의 GPT'를 만드는 전체 과정을 상세한 코드와 함께 안내해 드리겠습니다.

전체 과정은 아래와 같은 단계로 진행됩니다.

라이브러리 준비: 파인튜닝에 필요한 datasets 라이브러리를 추가합니다.
데이터 전처리: '노무현' 대통령의 연설문만 필터링하고, 모델 학습에 적합한 형태로 가공합니다.
모델 및 토크나이저 로딩: 사전 학습된 KoGPT2 모델과 토크나이저를 불러옵니다.
학습 설정: TrainingArguments를 통해 학습 파라미터(배치 사이즈, 학습률 등)를 설정합니다.
학습 실행: Trainer를 사용하여 실제 파인튜닝을 진행합니다.
모델 저장 및 테스트: 파인튜닝된 나만의 모델을 저장하고, 어떻게 문장을 생성하는지 테스트합니다.
```

In [1]:
import torch
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# --- 1. 라이브러리 준비 ---
# 파인튜닝에 Hugging Face의 datasets 라이브러리가 필요합니다.
# 터미널에서 아래 명령어로 설치해주세요.
# pdm add datasets

In [2]:
from president_speech.db.parquet_interpreter import read_parquet
df = read_parquet()
df.head(3)

,division_number,president,title,date,location,kind,speech_text
5368,1305368,박정희,제5대 대통령 취임식 대통령 취임사,1963.12.17,국내,취임사,"\n\n\n단군성조가 천혜의 이 강토 위에 국기를 닦으신지 반만년, 연면히 이어온 ..."
5369,1305369,박정희,국회 개회식 치사,1963.12.17,국내,기념사,"존경하는 국회의장, 의원제위 그리고 내외귀빈 여러분! 오늘 이 뜻깊은 제3공화국의..."
5370,1305370,박정희,신년 메시지,1964.01.01,국내,신년사,친애하는 국내외의 동포 여러분! 혁명의 고된 시련을 겪고 민정이양으로 매듭을 지은...


In [3]:
df_roh = df[df['president'] == '노무현'].copy()
df_roh
print(f"'노무현' 전 대통령 연설문 개수: {len(df_roh)}")

'노무현' 전 대통령 연설문 개수: 780


In [4]:
# 2-2. 모든 연설문을 하나의 텍스트로 결합
# 각 연설문 사이에 공백(또는 문단 구분자)을 두어 연결합니다.
# 모델이 문맥을 학습할 때 연설문이 끝났다는 것을 암시적으로 알 수 있도록 합니다.
combined_text = "\n\n".join(df_roh['speech_text'].tolist())

In [5]:
# 2-3. 데이터를 Hugging Face Dataset 형식으로 변환
# 텍스트 데이터를 딕셔너리 형태로 만들어 Dataset 객체를 생성합니다.
data_dict = {'text': [combined_text]}
raw_dataset = Dataset.from_dict(data_dict)

In [6]:
# --- 3. 모델 및 토크나이저 로딩 ---
MODEL_NAME = 'skt/kogpt2-base-v2'

# 3-1. 토크나이저 로딩
# KoGPT2 토크나이저는 bos, eos, pad 토큰이 특별히 지정되어 있지 않을 수 있습니다.
# 파인튜닝을 위해 명시적으로 지정해주는 것이 안정적입니다.
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    bos_token='<s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>'
)

In [8]:
# 3-2. 데이터셋 토크나이징 및 청킹(Chunking)
# 텍스트를 토큰 ID로 변환하고, 긴 텍스트를 일정한 길이(block_size)의 덩어리로 자릅니다.
def tokenize_and_chunk(examples):
    # 전체 텍스트를 토크나이징합니다.
    tokenized_output = tokenizer(examples['text'], truncation=False) # 긴 텍스트이므로 자르지 않음

    block_size = 128  # 한 번에 처리할 토큰의 수 (GPU 메모리에 따라 조절)
    
    # 결과를 저장할 딕셔너리
    result = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
    }

    # 모든 토큰을 하나로 합칩니다.
    concatenated_ids = sum(tokenized_output['input_ids'], [])
    total_length = len(concatenated_ids)

    # block_size 단위로 텍스트를 나눕니다.
    for i in range(0, total_length, block_size):
        chunk = concatenated_ids[i:i + block_size]
        
        # 마지막 청크가 block_size보다 작으면 패딩을 추가합니다.
        if len(chunk) < block_size:
            padding_length = block_size - len(chunk)
            chunk.extend([tokenizer.pad_token_id] * padding_length)
            
        result["input_ids"].append(chunk)
        # 패딩 토큰은 attention 계산에서 제외하기 위해 0으로 설정
        attention_mask = [1] * (len(chunk) - chunk.count(tokenizer.pad_token_id)) + [0] * chunk.count(tokenizer.pad_token_id)
        result["attention_mask"].append(attention_mask)
        # Causal LM 파인튜닝에서는 input_ids와 labels를 동일하게 설정합니다.
        # Trainer가 내부적으로 labels를 한 칸씩 밀어서(shift) 처리해줍니다.
        result["labels"].append(chunk)

    return result

In [9]:
# map 함수를 사용하여 전체 데이터셋에 함수를 적용합니다.
lm_dataset = raw_dataset.map(
    tokenize_and_chunk,
    batched=True,
    remove_columns=raw_dataset.column_names
)
print("파인튜닝을 위한 데이터셋 준비 완료!")
print(lm_dataset)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

파인튜닝을 위한 데이터셋 준비 완료!
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5743
})


In [10]:
# --- 4. 모델 로딩 및 학습 설정 ---

# 4-1. 사전 학습된 KoGPT2 모델 로딩
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
# 모델의 토큰 임베딩 사이즈를 토크나이저에 맞게 조절
model.resize_token_embeddings(len(tokenizer))

Embedding(51200, 768)

In [13]:
# 4-2. 학습 관련 인자(Argument) 설정
# 이 부분이 파인튜닝의 '조종간' 역할을 합니다.
training_args = TrainingArguments(
    output_dir='./my_roh_gpt2_results',  # 학습 결과물(체크포인트)이 저장될 디렉터리
    num_train_epochs=3,                  # 전체 데이터셋에 대한 학습 횟수 (에포크)
    per_device_train_batch_size=4,       # GPU 하나당 배치 사이즈 (메모리가 부족하면 1 또는 2로 줄이세요)
    learning_rate=5e-5,                  # 학습률 (일반적으로 1e-4 ~ 5e-5 사이를 사용)
    weight_decay=0.01,                   # 가중치 감소 (과적합 방지)
    logging_dir='./logs',                # 로그 저장 디렉터리
    logging_steps=10,                    # 몇 스텝마다 로그를 출력할지
    save_steps=500,                      # 몇 스텝마다 모델 체크포인트를 저장할지
    save_total_limit=2,                  # 최대 몇 개의 체크포인트를 저장할지
    fp16=True,                           # **중요**: RTX 20/30/40 시리즈 GPU 사용 시 학습 속도 및 메모리 효율을 크게 향상시킴
    report_to="none",                    # 외부 서비스(wandb 등)에 로그를 전송하지 않음
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

### PIP_EXTRA_INDEX_URL=https://download.pytorch.org/whl/cu11  pdm add "transformers[torch]" 

In [25]:
# 4-2. 학습 관련 인자(Argument) 설정
# 이 부분이 파인튜닝의 '조종간' 역할을 합니다.
training_args = TrainingArguments(
    output_dir='./my_roh_gpt2_results',  # 학습 결과물(체크포인트)이 저장될 디렉터리
    num_train_epochs=3,                  # 전체 데이터셋에 대한 학습 횟수 (에포크)
    per_device_train_batch_size=4,       # GPU 하나당 배치 사이즈 (메모리가 부족하면 1 또는 2로 줄이세요)
    learning_rate=5e-5,                  # 학습률 (일반적으로 1e-4 ~ 5e-5 사이를 사용)
    weight_decay=0.01,                   # 가중치 감소 (과적합 방지)
    logging_dir='./logs',                # 로그 저장 디렉터리
    logging_steps=10,                    # 몇 스텝마다 로그를 출력할지
    save_steps=500,                      # 몇 스텝마다 모델 체크포인트를 저장할지
    save_total_limit=2,                  # 최대 몇 개의 체크포인트를 저장할지
    fp16=True,                           # **중요**: RTX 20/30/40 시리즈 GPU 사용 시 학습 속도 및 메모리 효율을 크게 향상시킴
    report_to="none",                    # 외부 서비스(wandb 등)에 로그를 전송하지 않음
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

### pdm add -dG finetuning accelerate

In [11]:
training_args = TrainingArguments(
    output_dir='./my_roh_gpt2_results',  # 학습 결과물(체크포인트)이 저장될 디렉터리
    num_train_epochs=3,                  # 전체 데이터셋에 대한 학습 횟수 (에포크)
    per_device_train_batch_size=4,       # GPU 하나당 배치 사이즈 (메모리가 부족하면 1 또는 2로 줄이세요)
    learning_rate=5e-5,                  # 학습률 (일반적으로 1e-4 ~ 5e-5 사이를 사용)
    weight_decay=0.01,                   # 가중치 감소 (과적합 방지)
    logging_dir='./logs',                # 로그 저장 디렉터리
    logging_steps=10,                    # 몇 스텝마다 로그를 출력할지
    save_steps=500,                      # 몇 스텝마다 모델 체크포인트를 저장할지
    save_total_limit=2,                  # 최대 몇 개의 체크포인트를 저장할지
    fp16=True,                           # **중요**: RTX 20/30/40 시리즈 GPU 사용 시 학습 속도 및 메모리 효율을 크게 향상시킴
    report_to="none",                    # 외부 서비스(wandb 등)에 로그를 전송하지 않음
)

In [12]:
# 4-3. 데이터 콜레이터(Data Collator) 설정
# 배치(batch)를 만들 때 데이터를 어떻게 묶을지 정의합니다.
# MLM(Masked Language Modeling)을 False로 설정하여 Causal LM 파인튜닝을 진행합니다.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
# --- 5. 학습 실행 ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=data_collator,
)

print("="*50)
print("파인튜닝을 시작합니다. GPU 사양에 따라 시간이 소요될 수 있습니다.")
print("="*50)

# 학습 시작!
trainer.train()

print("파인튜닝 완료!")

파인튜닝을 시작합니다. GPU 사양에 따라 시간이 소요될 수 있습니다.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.155800
20,4.123700
30,3.873300
40,3.826400
50,3.812800
60,3.929700
70,3.859900
80,3.798600
90,3.613300
100,3.676400


파인튜닝 완료!


In [14]:
# --- 6. 모델 저장 및 테스트 ---

# 6-1. 최종 모델 저장
final_model_path = './my_roh_gpt2_final'
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"파인튜닝된 모델과 토크나이저가 '{final_model_path}'에 저장되었습니다.")

파인튜닝된 모델과 토크나이저가 './my_roh_gpt2_final'에 저장되었습니다.


In [15]:
# 6-2. 파인튜닝된 모델로 텍스트 생성 테스트
# 저장된 모델과 토크나이저를 다시 불러옵니다.
model_ft = AutoModelForCausalLM.from_pretrained(final_model_path)
tokenizer_ft = AutoTokenizer.from_pretrained(final_model_path)


In [16]:
# GPU를 사용하여 텍스트 생성
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft.to(device)

def generate_text(prompt, max_len=128):
    input_ids = tokenizer_ft.encode(prompt, return_tensors='pt').to(device)
    
    # beam search, top-k sampling 등 다양한 생성 전략을 사용할 수 있습니다.
    gen_ids = model_ft.generate(input_ids,
                               max_length=max_len,
                               repetition_penalty=2.0,
                               pad_token_id=tokenizer_ft.pad_token_id,
                               eos_token_id=tokenizer_ft.eos_token_id,
                               bos_token_id=tokenizer_ft.bos_token_id,
                               use_cache=True)
    
    generated_text = tokenizer_ft.decode(gen_ids[0])
    return generated_text

In [17]:
# 테스트 프롬프트
prompt1 = "국민 여러분, 우리나라의 미래는"
prompt2 = "특권과 반칙이 없는 사회를 만들기 위해서는"

print("\n--- 파인튜닝 모델 생성 테스트 ---")
print(f"입력: {prompt1}")
print(f"생성 결과: {generate_text(prompt1)}")
print("-" * 30)
print(f"입력: {prompt2}")
print(f"생성 결과: {generate_text(prompt2)}")
print("--- 테스트 완료 ---")


--- 파인튜닝 모델 생성 테스트 ---
입력: 국민 여러분, 우리나라의 미래는
생성 결과: 국민 여러분, 우리나라의 미래는 밝습니다. 그러나 아직 가야할 길이 많이 남아 있습니다. 무엇보다 우리 국민의 역량에 대한 믿음과 책임감을 가지고 미래를 준비해야 합니다.  저와 참여정부는 이 점에 대해서 무거운 책임을 느끼고 있습니다.
<unk>
참여정부가 출범한 지도 1년이 지났지만 아직도 많은 과제들이 남아있고, 또 남은 과제도 많습니다만, 그 중에서도 가장 중요한 것은 역시, 우리의 미래에 대해 낙관적인 전망을 가질 수 있는 역량을 갖추는 일이라고 생각합니다. 
<unk>
저는 취임 초부터 ‘국민과 함께하는 민주주의’, ‘대화와 타협’의 국정운영을 강조해 왔습니다.
그러나 지금 우리는 이러한 민주주의에 이르지 못하고 있고, 또한 선진국에 뒤처
------------------------------
입력: 특권과 반칙이 없는 사회를 만들기 위해서는
생성 결과: 특권과 반칙이 없는 사회를 만들기 위해서는 무엇보다 공정하고 투명한 경제시스템을 구축해야 합니다. 
<unk>
참여정부는 그동안 공정한 시장질서를 확립하기 위해 노력해 왔습니다. 정경유착과 관치금융의 폐해를 근절하고, 투명성과 책임성을 높이는 데 주력했습니다.
또한 금융실명제를 도입해서 기업의 지배구조를 개선하는 한편, 증권관련 집단소송제의 적용대상을 확대하였습니다만, 아직 미흡합니다. 앞으로도 우리 경제의 발목을 잡고 있는 불합리한 규제는 과감히 철폐하여 나갈 것입니다. 이를 통해 기업지배구조, 내부거래와 같은 불공정 거래는 물론, 시장의 감시기능을 한층 강화할 것입니다.
<unk>
이와 함께
--- 테스트 완료 ---


In [18]:
def prompt(text):
    """
    파인튜닝된 모델로 텍스트를 생성하는 함수입니다.
    :param text: 입력 프롬프트
    :return: 생성된 텍스트
    """
    print("\n--- 파인튜닝 모델 생성 테스트 ---")
    print(f"입력: {text}")
    print(f"생성 결과: {generate_text(text)}")

In [19]:
prompt("북한과의 관계")


--- 파인튜닝 모델 생성 테스트 ---
입력: 북한과의 관계
생성 결과: 북한과의 관계 개선에 있어서도 매우 중요한 역할을 할 것입니다.  나는 이번 방문을 계기로 한, 미, 일 3국이 더욱 긴밀히 협력해 나가기로 합의했습니다. 특히 북핵문제의 평화적 해결을 위한 공동노력에 관해서도 의견을 같이 했습니다.
<unk>
이번 정상회담은 양국간 실질협력을 확대하는 좋은 계기가 될 것으로 기대합니다. 아울러 우리 두 나라가 유엔을 비롯한 국제무대에서도 긴밀한 공조를 유지해 나갈 것임을 다시 한번 확인하였습니다만, 저는 이번에 합의한 사항들을 하나하나 실천해서 앞으로 더 많은 성과를 거두도록 노력하겠으며, 여러분의 적극적인 협력을 당부드립니다.
 
<unk>
존경하고 경청해주셔
